# Metrics computation
In this notebook, we compute F1 score metrics for both GRU and BART based models for KBQA.

## imports

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

## Loading answers

In [3]:
tset = pd.read_json('./val2.json')
tset.head()

,question,choices,program,sparql,answer
0,Is Socceroos the Twitter username of the perso...,"[yes, no, unknown, unknown, unknown, unknown, ...","[{'function': 'FindAll', 'dependencies': [], '...",ASK { ?e <pred:instance_of> ?c . ?c <pred:name...,no
1,Secret of the Wings is related in what way to ...,"[symptoms, successful candidate, coextensive w...","[{'function': 'Find', 'dependencies': [], 'inp...","SELECT DISTINCT ?p WHERE { ?e_1 <pred:name> ""S...",film release region
2,Which TOID applies to the capital city of Norf...,"[4000000074580405, 4000000074553344, 400000007...","[{'function': 'Find', 'dependencies': [], 'inp...",SELECT DISTINCT ?pv WHERE { ?e <pred:instance_...,4000000074573264
3,For which work was Mary Elizabeth Winstead nom...,"[Angels with Dirty Faces, Purple Rain, Ed Wood...","[{'function': 'Find', 'dependencies': [], 'inp...",SELECT DISTINCT ?qpv WHERE { ?e_1 <pred:name> ...,Bobby
4,"Where was a visual artwork, that can be found ...","[Westwood, Grauman's Chinese Theatre, European...","[{'function': 'FindAll', 'dependencies': [], '...",SELECT DISTINCT ?qpv WHERE { ?e <pred:instance...,Germany


In [20]:
qAndA = tset[['question','answer']]

In [21]:
qAndA

,question,answer
0,Is Socceroos the Twitter username of the perso...,no
1,Secret of the Wings is related in what way to ...,film release region
2,Which TOID applies to the capital city of Norf...,4000000074573264
3,For which work was Mary Elizabeth Winstead nom...,Bobby
4,"Where was a visual artwork, that can be found ...",Germany
...,...,...
4995,"Where was a movie, titled Raging Bull, publish...",Germany
4996,"Which language is used at www.8-mile.com, offi...",English
4997,Among the states of the United States that sha...,Idaho
4998,How many males with a given name have the Cave...,1


In [29]:
f = open("q&a.txt", "a")

for i in range(len(qAndA)):
    f.write(str(i)+': '+str(qAndA.iloc[i]['question'])+' :: '+str(qAndA.iloc[i]['answer'])+'\n'+'\n')


f.close()

In [4]:
list(tset['answer'].iloc[:1000])

['no',
 'film release region',
 '4000000074573264',
 'Bobby',
 'Germany',
 'famous people',
 'English',
 'Michael Hordern',
 'Reese Witherspoon',
 'Gone with the Wind',
 'Sridevi',
 'John Byrne',
 '2003',
 'no',
 'The Day After Tomorrow',
 '2015-07-01',
 '2',
 'yes',
 'cause of death',
 'Bewitched',
 'David O. Russell',
 'yes',
 'Province of Turin',
 'Montgomery County',
 'no',
 'Enrico Fermi',
 'http://www.waterforelephants.dk/',
 'The House Bunny',
 '1864-08-28',
 'Peter Jackson',
 'Flags of Our Fathers',
 '50',
 'yes',
 'no',
 '1983',
 'Jermaine Jackson',
 'Riverside County',
 'Naomi Watts',
 '2',
 '5',
 'Uganda',
 'English',
 'Englewood',
 'occupation',
 '1995',
 '31st Academy Awards',
 'English',
 'Allen County',
 '1984',
 '0000 0000 4096 9255',
 '3',
 '1546-11-04',
 'http://www.cbs.com/primetime/two_and_a_half_men',
 'instrument',
 'Fashion Institute of Technology',
 'Goa',
 'Canterbury',
 '30 minute',
 'cast member',
 'FIFA World Rankings',
 'yes',
 '17',
 'Henry James',
 'Aaron

## GRU

#### loading predictions

In [6]:
predictionsGru = []
with open('../predictions/gru/predictGRU.txt', 'r') as file:
    for i,line in enumerate(file):
        predictionsGru.append(str(i)+': '+line.strip())

In [7]:
predictionsGru

['0: no',
 '1: None',
 '2: None',
 '3: None',
 '4: None',
 '5: None',
 '6: None',
 '7: None',
 '8: William T. Orr',
 '9: None',
 '10: Sridevi',
 '11: None',
 '12: None',
 '13: None',
 '14: The Day After Tomorrow',
 '15: None',
 '16: 0',
 '17: no',
 '18: None',
 '19: None',
 '20: Joseph Conrad',
 '21: yes',
 '22: None',
 '23: None',
 '24: no',
 '25: None',
 '26: http://www.waterforelephants.dk/',
 '27: The Expendables 2',
 '28: None',
 '29: Lara Flynn Boyle',
 '30: Flags of Our Fathers',
 '31: 0',
 '32: None',
 '33: no',
 '34: 1983',
 '35: Sherri Shepherd',
 '36: None',
 '37: None',
 '38: 1',
 '39: 0',
 '40: Uganda',
 '41: None',
 '42: Englewood',
 '43: None',
 '44: None',
 '45: None',
 '46: English',
 '47: Allen County',
 '48: None',
 '49: None',
 '50: None',
 '51: None',
 '52: None',
 '53: None',
 '54: None',
 '55: Goa',
 '56: None',
 '57: 30 minute',
 '58: None',
 '59: None',
 '60: no',
 '61: None',
 '62: None',
 '63: None',
 '64: None',
 '65: None',
 '66: 0',
 '67: 4',
 '68: None',


## BART

In [8]:
predictionsBART = []
with open('../predictions/bart/predict.txt', 'r') as file:
    for i,line in enumerate(file):
        predictionsBART.append(str(i)+': '+line.strip())

## write to files

In [11]:
with open('../predictions/bart/BARTpredict.txt','w') as file:
    for answer in predictionsBART:
        file.write(answer+'\n\n')

In [12]:
with open('../predictions/bart/GRUpredict.txt','w') as file:
    for answer in predictionsGru:
        file.write(answer+'\n\n')

In [9]:
predictionsBART

['0: no',
 '1: film release region',
 '2: 4000000074573264',
 '3: Bobby',
 '4: None',
 '5: famous people',
 '6: None',
 '7: Michael Hordern',
 '8: Reese Witherspoon',
 '9: Gone with the Wind',
 '10: Sridevi',
 '11: John Byrne',
 '12: 2003',
 '13: no',
 '14: The Day After Tomorrow',
 '15: 2015-07-01',
 '16: 2',
 '17: yes',
 '18: cause of death',
 '19: Bewitched',
 '20: David O. Russell',
 '21: yes',
 '22: no',
 '23: Montgomery County',
 '24: no',
 '25: Enrico Fermi',
 '26: http://www.waterforelephants.dk/',
 '27: The House Bunny',
 '28: 1864-08-28',
 '29: Peter Jackson',
 '30: Flags of Our Fathers',
 '31: 50',
 '32: George Benson',
 '33: no',
 '34: 1983',
 '35: Jermaine Jackson',
 '36: Riverside County',
 '37: Naomi Watts',
 '38: 2',
 '39: 5',
 '40: Uganda',
 '41: English',
 '42: Englewood',
 '43: occupation',
 '44: 1995',
 '45: Separate Tables',
 '46: English',
 '47: Allen County',
 '48: None',
 '49: 0000 0000 4096 9255',
 '50: 3',
 '51: 1546-11-04',
 '52: http://www.cbs.com/primetime/

In [10]:
len(predictionsBART)

5000

## Compute metrics

In [ ]:
def compute_scores();

In [10]:
ytrueList = list(map(lambda x: x.strip(),list(tset['answer'])))



In [17]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
print('F1 BART: ',f1_score(ytrueList, predictionsBART, average='macro'))
print('F1 GRU: ',f1_score(ytrueList, predictionsGru, average='macro'))


print('ACC BART: ',accuracy_score(ytrueList, predictionsBART, normalize=True))
print('ACC GRU',accuracy_score(ytrueList, predictionsGru, normalize=True))


F1 BART:  0.8960749783808899
F1 GRU:  0.2028614670130376
ACC BART:  0.91
ACC GRU 0.1928
